In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2, os
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = "../Models/best_fracture_model.keras"   # update if your path is different
model = tf.keras.models.load_model(model_path)

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [ ]:
# Choose your test image
test_image_path = r"C:\Users\lenovo\Desktop\FractureNet+\Dataset\"

# Load image and preprocess
img = image.load_img(test_image_path, target_size=(224,224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)[0][0]
label = "🩻 Fractured" if prediction > 0.5 else "✅ Non-Fractured"

# Display
plt.imshow(image.load_img(test_image_path))
plt.title(f"Prediction: {label}\nConfidence: {prediction:.2f}")
plt.axis('off')
plt.show()


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap.numpy()


In [ ]:
# Identify last conv layer automatically
last_conv_layer_name = [layer.name for layer in model.layers if 'conv' in layer.name][-1]

# Generate Grad-CAM heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Load original image
img = cv2.imread(test_image_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Overlay
superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

# Display Grad-CAM
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.title(f"Grad-CAM Visualization: {label}")
plt.axis('off')
plt.show()


In [ ]:
import glob

test_folder = r"C:\Users\lenovo\BoneAI_Project\test_images"

for img_path in glob.glob(os.path.join(test_folder, "*.jpg")):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0][0]
    label = "Fractured" if pred > 0.5 else "Non-Fractured"
    print(f"{os.path.basename(img_path)} → {label} ({pred:.2f})")
